# generate_error_table

## imports

In [8]:
import pandas as pd
import numpy as np

## load data

In [9]:
df0 = pd.read_csv('data/benchmarks_20230122.csv')
df1 = pd.read_csv('data/benchmarks_20230126.csv')
df2 = pd.read_csv('data/equiwidth.csv')
df1 = df1.fillna('TPC-DS')
df = pd.concat([df0, df1, df2], axis=0)

## generate errors

In [10]:
def generate_errors(df):
    df = df.copy()  # Convenience
    
    df['absolute_error'] = np.absolute(df['estimated_output'] - df['real_output'])
    df['relative_error'] = df['absolute_error'] / df['real_output']
    df['x'] = df['estimated_output'] / df['real_output']
    df['1/x'] = 1 / df['x']
    df['q_error'] = df[['x','1/x']].max(axis=1)
    df['real_output_lower_bound'] = df['real_output']
    df['real_output_lower_bound'] = df['real_output_lower_bound'].clip(1)
    df['estimated_output_lower_bound'] = df['estimated_output']
    df['estimated_output_lower_bound'] = df['estimated_output_lower_bound'].clip(1)
    df['pseudo_x'] = df['estimated_output_lower_bound'] / df['real_output_lower_bound']
    df['pseudo_1/x'] = 1 / df['pseudo_x']
    df['pseudo_q_error'] = df[['pseudo_x','pseudo_1/x']].max(axis=1)
    df['mean_squared_error'] = df['absolute_error'] ** 2
    df.drop(['x', '1/x', 'real_output_lower_bound', 'estimated_output_lower_bound', 'pseudo_x', 'pseudo_1/x'], axis=1, inplace = True)

    return df

## result

### with_NULLs

In [11]:
df_with_errors = generate_errors(df)
df_with_errors

df_with_errors.groupby(["operator_type",
                        "benchmark",
                        "histogram"]).agg(root_mean_squared_error=("mean_squared_error", lambda x: np.sqrt(np.mean(x))),
                                          mean_absolute_error=("absolute_error", np.mean),
                                          mean_relative_error=("relative_error", np.mean),
                                          mean_q_error=("q_error", np.mean),
                                          mean_pseudo_q_error=("pseudo_q_error", np.mean))

root_mean_squared_error  \
operator_type benchmark histogram                                              
Aggregate     TPC-DS    EqualDistinctCountHistogram             17989.000000   
                        EquiHeightHistogram                     17989.000000   
                        MaxDiffFrequencyHistogram               17989.000000   
              TPC-H     EqualDistinctCountHistogram             45175.000000   
                        EquiHeightHistogram                     45175.000000   
                        EquiWidthHistogram                      45175.000000   
                        MaxDiffFrequencyHistogram               45175.000000   
JoinHashLeft  JoinOrder EqualDistinctCountHistogram            559261.272399   
                        EquiHeightHistogram                    612721.251615   
                        EquiWidthHistogram                     616046.398980   
                        MaxDiffFrequencyHistogram              618620.030724   
              TPC-DS    EqualDistinctCountHistogram            162617.475813   
                        EquiHeightHistogram                    162040.388937   
                        MaxDiffFrequencyHistogram              162431.555619   
              TPC-H     EqualDistinctCountHistogram              9438.560837   
                        EquiHeightHistogram                      9544.760094   
                        EquiWidthHistogram                       9521.738899   
                        MaxDiffFrequencyHistogram                9489.153474   
JoinHashRight JoinOrder EqualDistinctCountHistogram            158918.703744   
                        EquiHeightHistogram                    363615.335560   
                        EquiWidthHistogram                     365814.315299   
                        MaxDiffFrequencyHistogram              339846.181934   
              TPC-DS    EqualDistinctCountHistogram            327742.859216   
                        EquiHeightHistogram                    321337.180889   
                        MaxDiffFrequencyHistogram              327490.352136   
              TPC-H     EqualDistinctCountHistogram              6258.764706   
                        EquiHeightHistogram                      5722.389985   
                        EquiWidthHistogram                       5708.962940   
                        MaxDiffFrequencyHistogram                6187.891527   
TableScan     JoinOrder EqualDistinctCountHistogram            194434.185805   
                        EquiHeightHistogram                    136659.561976   
                        EquiWidthHistogram                     138709.189727   
                        MaxDiffFrequencyHistogram              137920.008784   
              TPC-DS    EqualDistinctCountHistogram              2065.867761   
                        EquiHeightHistogram                      2065.718074   
                        MaxDiffFrequencyHistogram                2077.293380   
              TPC-H     EqualDistinctCountHistogram              4037.001665   
                        EquiHeightHistogram                       503.745038   
                        EquiWidthHistogram                       4031.817534   
                        MaxDiffFrequencyHistogram                1591.079499   

                                                     mean_absolute_error  \
operator_type benchmark histogram                                          
Aggregate     TPC-DS    EqualDistinctCountHistogram         17989.000000   
                        EquiHeightHistogram                 17989.000000   
                        MaxDiffFrequencyHistogram           17989.000000   
              TPC-H     EqualDistinctCountHistogram         45175.000000   
                        EquiHeightHistogram                 45175.000000   
                        EquiWidthHistogram                  45175.000000   
                        MaxDiffFrequencyHistogram           45175.000000   
JoinHash

### without_NULLs

In [12]:
size_before = len(df)
df = df.drop(df[df.real_output == 0].index)
print(f"Removed {len(df)-size_before} ({(size_before-len(df))/size_before:.2%}) rows because the 'real_output' cell was zero.")
df = df.drop(df[df.estimated_output == 0].index)
print(f"Removed {len(df)-size_before} ({(size_before-len(df))/size_before:.2%}) rows because the 'estimated_output' cell was zero.")

df_with_errors = generate_errors(df)

df_with_errors.groupby(["operator_type",
                        "benchmark",
                        "histogram"]).agg(root_mean_squared_error=("mean_squared_error", lambda x: np.sqrt(np.mean(x))),
                                          mean_absolute_error=("absolute_error", np.mean),
                                          mean_relative_error=("relative_error", np.mean),
                                          mean_q_error=("q_error", np.mean),
                                          mean_pseudo_q_error=("pseudo_q_error", np.mean))

Removed -228 (4.16%) rows because the 'real_output' cell was zero.
Removed -723 (13.20%) rows because the 'estimated_output' cell was zero.


root_mean_squared_error  \
operator_type benchmark histogram                                              
Aggregate     TPC-DS    EqualDistinctCountHistogram             17989.000000   
                        EquiHeightHistogram                     17989.000000   
                        MaxDiffFrequencyHistogram               17989.000000   
              TPC-H     EqualDistinctCountHistogram             45175.000000   
                        EquiHeightHistogram                     45175.000000   
                        EquiWidthHistogram                      45175.000000   
                        MaxDiffFrequencyHistogram               45175.000000   
JoinHashLeft  JoinOrder EqualDistinctCountHistogram            604033.347385   
                        EquiHeightHistogram                    660365.180598   
                        EquiWidthHistogram                     668380.944022   
                        MaxDiffFrequencyHistogram              654033.382324   
              TPC-DS    EqualDistinctCountHistogram            168808.427838   
                        EquiHeightHistogram                    172927.702508   
                        MaxDiffFrequencyHistogram              164200.436606   
              TPC-H     EqualDistinctCountHistogram              1528.397473   
                        EquiHeightHistogram                      1581.123368   
                        EquiWidthHistogram                       1567.818720   
                        MaxDiffFrequencyHistogram                1811.417638   
JoinHashRight JoinOrder EqualDistinctCountHistogram            171649.586038   
                        EquiHeightHistogram                    386650.251862   
                        EquiWidthHistogram                     373058.866412   
                        MaxDiffFrequencyHistogram              367075.812845   
              TPC-DS    EqualDistinctCountHistogram            332262.802732   
                        EquiHeightHistogram                    330650.361717   
                        MaxDiffFrequencyHistogram              332006.811967   
              TPC-H     EqualDistinctCountHistogram              1513.844382   
                        EquiHeightHistogram                      1003.228620   
                        EquiWidthHistogram                       1003.368256   
                        MaxDiffFrequencyHistogram                1096.439252   
TableScan     JoinOrder EqualDistinctCountHistogram            196000.408539   
                        EquiHeightHistogram                    123850.717266   
                        EquiWidthHistogram                     124207.115301   
                        MaxDiffFrequencyHistogram              122993.561323   
              TPC-DS    EqualDistinctCountHistogram              2084.547551   
                        EquiHeightHistogram                      2090.779162   
                        MaxDiffFrequencyHistogram                2145.341320   
              TPC-H     EqualDistinctCountHistogram              4103.733080   
                        EquiHeightHistogram                       530.990405   
                        EquiWidthHistogram                       4098.463254   
                        MaxDiffFrequencyHistogram                1618.278344   

                                                     mean_absolute_error  \
operator_type benchmark histogram                                          
Aggregate     TPC-DS    EqualDistinctCountHistogram         17989.000000   
                        EquiHeightHistogram                 17989.000000   
                        MaxDiffFrequencyHistogram           17989.000000   
              TPC-H     EqualDistinctCountHistogram         45175.000000   
                        EquiHeightHistogram                 45175.000000   
                        EquiWidthHistogram                  45175.000000   
                        MaxDiffFrequencyHistogram           45175.000000   
JoinHash

## compare_operators

In [14]:
grouped = df.groupby(["benchmark", "histogram", "query", "operator_type"]).size()

queries = df[["benchmark", "query"]].drop_duplicates()

# super inefficient, doesn't matter for now
op_counts = {}
for _, outer_benchmark, outer_query in queries.itertuples():
    for (_1, benchmark, histogram, query, operator_type, count) in grouped.reset_index().itertuples():
        if outer_benchmark == benchmark and outer_query == query:
            if (benchmark, query, operator_type) not in op_counts:
                op_counts[(benchmark, query, operator_type)] = count
                continue

            if op_counts[(benchmark, query, operator_type)] != count:
                print(f"WARNING: different value of {count} (previously {op_counts[(benchmark, query, operator_type)]}) for {benchmark}, query {query}, op {operator_type}")
